# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/blob/master/Exercises/moviegenre.png?raw=1"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 




### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

# TRANSFORMACIÓN DATOS

In [ ]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.


In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
import re
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open('/content/drive/My Drive//glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [ ]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [ ]:
dataTraining["plot_vf"] = dataTraining.iloc[:,1].map(str) + " " + dataTraining.iloc[:,2].map(str)
dataTesting["plot_vf"] = dataTesting.iloc[:,1].map(str) + " " + dataTesting.iloc[:,2].map(str)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

def lematizacion(text):
  lmtzr = WordNetLemmatizer()
  VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
  ADJ_CODES = {'JJ','JJR','JJS'}
  ADV_CODES = {'RB','RBR','RBS'}
  NOUN_CODES = {'NN', 'NNP','NNS','NNPS'}
  new_text = []
  words = nltk.word_tokenize(text)
  role = nltk.pos_tag(words)
  
  for i,word in enumerate(words):
    if role[i][1] in VERB_CODES:
      new_word = lmtzr.lemmatize(word, 'v')
      #x =  TreebankWordDetokenizer().detokenize(new_word)
    else:
      if role[i][1] in ADJ_CODES:
        new_word = lmtzr.lemmatize(word, 'a')
        #x =  TreebankWordDetokenizer().detokenize(new_word)
      else:
        if role[i][1] in ADV_CODES:
          new_word = lmtzr.lemmatize(word, 'r')
          #x =  TreebankWordDetokenizer().detokenize(new_word)
        else:
          if role[i][1] in NOUN_CODES:
            new_word = lmtzr.lemmatize(word, 'n')
            #x =  TreebankWordDetokenizer().detokenize(new_word)
          else:
            new_word = lmtzr.lemmatize(word)
            #x =  TreebankWordDetokenizer().detokenize(new_word)
    if len(new_word) > 1:
      new_text.append(new_word)
  final_text = ' '.join(new_text)
  return final_text

In [ ]:
dataTraining["plot_vf"] = dataTraining["plot_vf"].apply(lematizacion)

In [ ]:
dataTesting["plot_vf"] = dataTesting["plot_vf"].apply(lematizacion)

In [ ]:
#from nltk.corpus import stopwords
#stop_words = stopwords.words('english')

#dataTraining["plot"] = dataTraining["plot"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#dataTesting["plot"] = dataTesting["plot"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [ ]:
plot1=dataTraining['plot_vf']
plot2=dataTesting['plot_vf']

In [ ]:
A1=plot1.to_list()
A2=plot2.to_list()
A3=A1+A2

In [ ]:
import string
voca3 =[''.join(c for c in s if c not in string.punctuation) for s in A3]
voca1 =[''.join(c for c in s if c not in string.punctuation) for s in A1]
voca2=[''.join(c for c in s if c not in string.punctuation) for s in A2]

In [ ]:
X=[x.split() for x in voca1]
len_=pd.Series([len(x) for x in X])
len_.describe(percentiles=[.8,.9,.95,.975,.99]) 

count    7895.000000
mean      122.221406
std        80.242849
min         3.000000
50%       106.000000
80%       173.000000
90%       230.000000
95%       279.000000
97.5%     318.650000
99%       367.000000
max      1648.000000
dtype: float64

In [ ]:
MAX_SEQUENCE_LENGTH = 370

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(voca1)
sequences = tokenizer.texts_to_sequences(voca1)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 34074 unique tokens.


In [ ]:
X_trainG, X_testG, y_train_genresG, y_test_genresG = train_test_split(data, y_genres, test_size=0.2, random_state=42)

In [ ]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


# DEFINICION DEL MODELO

## cargar librerias

In [ ]:
!pip install livelossplot

In [ ]:
!pip install bpmll

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from bpmll.bpmll import bp_mll_loss
from keras.layers import Flatten
import tensorflow as tf

## ejemplo kaggle Glove 1

> Bloc con sangría



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import tensorflow as tf
import keras

optimizador = keras.optimizers.Adam(lr=0.1) #0.001   

model = Sequential()
model.add(Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))
model.add(Dropout(0.3))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(LSTM(100))
model.add(Dense(24, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizador, metrics=[tf.keras.metrics.AUC()])

In [ ]:
history = model.fit(X_trainG, y_train_genresG, batch_size=128, epochs=50, verbose=1, validation_split=0.1)

Train on 5684 samples, validate on 632 samples
Epoch 1/50
5684/5684 [==============================] - 10s 2ms/step - loss: 0.3448 - auc: 0.6971 - val_loss: 0.3152 - val_auc: 0.7362
Epoch 2/50
5684/5684 [==============================] - 9s 1ms/step - loss: 0.3074 - auc: 0.7457 - val_loss: 0.3085 - val_auc: 0.7514
Epoch 3/50
5684/5684 [==============================] - 9s 2ms/step - loss: 0.3073 - auc: 0.7543 - val_loss: 0.3068 - val_auc: 0.7571
Epoch 4/50
5684/5684 [==============================] - 9s 2ms/step - loss: 0.3069 - auc: 0.7594 - val_loss: 0.3176 - val_auc: 0.7602
Epoch 5/50
5684/5684 [==============================] - 9s 2ms/step - loss: 0.3079 - auc: 0.7608 - val_loss: 0.3156 - val_auc: 0.7616
Epoch 6/50
5684/5684 [==============================] - 9s 2ms/step - loss: 0.3079 - auc: 0.7622 - val_loss: 0.3135 - val_auc: 0.7629
Epoch 7/50
5684/5684 [==============================] - 9s 2ms/step - loss: 0.3096 - auc: 0.7631 - val_loss: 0.3090 - val_auc: 0.7635
Epoch 8/50
568

In [ ]:
y_pred_genres = model.predict(X_testG)
roc_auc_score(y_test_genresG, y_pred_genres, average='macro')

In [ ]:
sequences1 = tokenizer.texts_to_sequences(voca2)
data1 = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict(data1)

In [ ]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,p_Film-Noir,p_History,p_Horror,p_Music,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.035560,0.043208,0.003329,0.003074,0.225789,0.072154,0.000290,0.513078,0.040691,0.095721,0.041303,0.002104,0.166987,0.002951,0.014640,0.200472,0.000195,0.485511,0.007209,0.004866,0.000615,0.345063,0.004908,0.023391
4,0.117986,0.010904,0.000209,0.027202,0.108843,0.711218,0.003455,0.775623,0.001001,0.002977,0.029711,0.010819,0.023189,0.009890,0.001935,0.065153,0.000793,0.053790,0.003842,0.005544,0.002227,0.659664,0.010013,0.034030
5,0.096273,0.006461,0.000051,0.087936,0.014675,0.884608,0.010657,0.943896,0.000715,0.002871,0.274023,0.033222,0.005776,0.002375,0.001271,0.537351,0.000794,0.230259,0.006587,0.000577,0.003261,0.887100,0.006831,0.007255
6,0.307982,0.183428,0.001471,0.009461,0.041886,0.042576,0.002071,0.713107,0.004486,0.021531,0.002604,0.026932,0.019063,0.000131,0.000716,0.162462,0.000327,0.067342,0.255511,0.000792,0.000630,0.624968,0.049115,0.005090
7,0.024649,0.031944,0.001513,0.001774,0.102751,0.022353,0.001047,0.346253,0.005952,0.071477,0.004157,0.001288,0.954867,0.000839,0.001122,0.389570,0.000129,0.056675,0.246069,0.001518,0.000022,0.671895,0.000690,0.000829


In [ ]:
res.to_csv('/content/drive/My Drive/Colab Notebooks/pred_genres_text_RF_V_9_52.csv', index_label='ID')

## Prueba XGBoost

> Bloc con sangría

In [ ]:
from xgboost import XGBClassifier
clf = OneVsRestClassifier(XGBClassifier(max_depth=4,n_estimators=200,n_jobs=-1))

In [ ]:
clf.fit(X_trainG, y_train_genresG)

In [ ]:
y_pred_genres = clf.predict(X_testG)
roc_auc_score(y_test_genresG, y_pred_genres, average='macro')